In [1]:
import itertools
import matplotlib.pyplot as plt
from constants import G
import numpy as np
import matplotlib.animation as animation
import json

In [2]:
class Planet:
    def __init__(self, mass, position, velocity, name):
        self.name = name
        self._mass = mass
        self._position = position
        self._acceleration = np.array([0.0, 0.0])
        self._force = np.array([0.0, 0.0])
        self._velocity = velocity
        self.forces = []
        self.dt = 60*60*24*7
        self.positions = [tuple(self._position)]

    @property
    def position(self):
        return self._position

    @position.setter
    def position(self, new_position):
        if not isinstance(new_position, np.ndarray):
            raise AssertionError('Position must be a element tuple.')
        if len(new_position) != 2:
            raise AssertionError('Position must contain 2 elements for x and y axis.')
        self._position = new_position

    @property
    def acceleration(self):
        return self._acceleration

    @acceleration.setter
    def acceleration(self, new_acceleration):
        self._acceleration = new_acceleration

    @property
    def force(self):
        return self._force

    @force.setter
    def force(self, new_force):
        self._force = new_force

    @property
    def mass(self):
        return self._mass

    @mass.setter
    def mass(self, new_mass):
        self._mass = new_mass

    @property
    def velocity(self):
        return self._velocity

    @velocity.setter
    def velocity(self, new_velocity):
        self._velocity = new_velocity

    def calculate_position(self):
        self.force = np.add.reduce(self.forces)
        a = self.calculate_acceleration(self.force)
        v = self.calculate_velocity(a)
        self.velocity = v
        ds = self.calculate_distance(self.velocity, a)
        self.position += ds
        # append the new position, used for plotting trajectory
        self.positions.append(tuple(self.position))
        self.forces.clear()

    def calculate_acceleration(self, force):
        a = force / self.mass
        return a

    def calculate_velocity(self, acceleration):
        # konst. rychlost na intervalu dt
        dv = acceleration * self.dt  # zmena rychlosti behem dt
        v = self.velocity + dv
        return v

    def calculate_distance(self, velocity, acceleration):
        ds = velocity * self.dt
        return ds

In [3]:
class SolarSystem:
    def __init__(self):
        self.planets = []

    def distance(self, pos1, pos2):
        return pos2 - pos1

    def unit_vector(self, vector):
        return vector / np.linalg.norm(vector)

    def add_planets(self, planets):
        for planet_name in planets:
            self.planets.append(
                Planet(position=np.array(planets[planet_name]['position']),
                       velocity=np.array(planets[planet_name]['velocity']),
                       mass=planets[planet_name]['mass'], name=planet_name))

    def calculate_force(self, planet1, planet2):
        F = (self.unit_vector(self.distance(planet1.position, planet2.position)) * (G * (planet1.mass * planet2.mass))) / (
                    np.linalg.norm(self.distance(planet1.position, planet2.position)) ** 2)
        return F


    def update_position(self):
        # do for each of two planets
        # mozna pomoci itertools?
        for index1, planet1 in enumerate(self.planets):
            for index2, planet2 in enumerate(self.planets):
                if index2 == index1:
                    continue
                else:
                    # print(planet1, planet2)
                    F = self.calculate_force(planet1, planet2)
                    planet1.forces.append(F)

        for planet in self.planets:
            planet.calculate_position()

        x = np.array([p.position[0] for p in self.planets])
        y = np.array([p.position[1] for p in self.planets])
        # print(self.planets[1].positions)
        # positions = np.array([planet.positions for planet in self.planets])
        # print(f"positions: {[x for x in positions]}")
        # x = np.array([position[0] for position in positions])
        # y = np.array([position[1] for position in positions])
        return x, y


In [4]:
class Animation:

    def __init__(self, solar_system):
        self.system = solar_system

    # Define the update function for the animation
    def update(self, frame):
        # Calculate the new positions of the planets
        x, y = self.system.update_position()
        # Update the positions in the plot graph
        self.plotting_graph.set_offsets(np.column_stack((x, y)))
        # Return the point object
        return self.plotting_graph,

    def plot(self):

        # Create the animation object
        na = 13
        self.fig, self.ax = plt.subplots()
        self.ax.set_xlim(-10**na, 10**na)
        self.ax.set_ylim(-10**na, 10**na)
        self.x = np.array([])
        self.y = np.array([])
        self.plotting_graph = self.ax.scatter([], [])

        self.animation = animation.FuncAnimation(self.fig, self.update, frames=60, interval=5)
        self.paused = False

        self.fig.canvas.mpl_connect('button_press_event', self.toggle_pause)

        # Show the plot
        plt.show()
        # FFwriter = animation.FFMpegWriter(codec='avi')
        # self.animation.save("video.gif")
        # writervideo = animation.FFMpegWriter(fps=60)
        # self.animation.save('increasingStraightLine.mp4', writer=writervideo)

    def toggle_pause(self, *args, **kwargs):
        if self.paused:
            self.animation.resume()
        else:
            self.animation.pause()
        self.paused = not self.paused


In [5]:
class Loader:
    def __init__(self, path):
        self.path = path

    def load_data(self):
        with open(self.path) as file:
            data = json.loads(file.read())
        return data

In [9]:
class Manager:
    def __init__(self, path):
        self.loader = Loader(path)
        self.system = SolarSystem()
        self.animation = Animation(self.system)

    def run(self):
        planets = self.loader.load_data()
        self.system.add_planets(planets)
        self.animation.plot()

In [10]:
# tk backend opens a new interactive window
%matplotlib tk
# run the calculations
manager = Manager("data/planets.json")
manager.run()